タスク
1. データは評価対象のもののみに絞る
2. 白川、大谷池の一部データを変形する
3. 水位データを日付ごとのデータに変換。その際に最大値、最小値、平均値、標準偏差、0時の値、23時の値を算出
4. 過去3日分の値をくっつける（5日分も作成）
5. １時間ごとのデータも2の修正を反映する

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path = '../data/'

# 雨量のデータの読み込み
rain_data = pd.read_csv(path+'rainfall/data.csv')
rain_stations = pd.read_csv(path+'rainfall/stations.csv')
print('shapeの確認')
print('雨量観測データ: ', rain_data.shape)
print('雨量観測データ所: ', rain_stations.shape)

# # 潮位のデータの読み込み
# tide_data = pd.read_csv(path+'tidelevel/data.csv')
# tide_stations = pd.read_csv(path+'tidelevel/stations.csv')
# print('潮位観測データ: ', tide_data.shape)
# print('潮位観測データ所: ', tide_stations.shape)

# 水位のデータの読み込み
water_data = pd.read_csv(path+'waterlevel/data.csv')
water_stations = pd.read_csv(path+'waterlevel/stations.csv')
print('水位観測データ: ', water_data.shape)
print('水位観測データ所: ', water_stations.shape)

# # damデータの読み込み
# dam = pd.read_csv(path+'dam.csv')
# print('damデータ: ', dam.shape)

水位観測データ:  (392066, 27)
水位観測データ所:  (180, 12)


In [5]:
water_data.head()

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
0,0,釜ヶ原,小瀬川,0.76,0.76,0.76,0.76,0.76,0.76,0.76,...,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76
1,0,下ヶ原(国),小瀬川,0.71,0.71,0.71,0.71,0.71,0.71,0.71,...,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71
2,0,防鹿(国),小瀬川,4.71,4.71,4.7,4.7,4.71,4.71,4.71,...,4.7,4.7,4.71,4.7,4.7,4.7,4.71,4.7,4.7,4.7
3,0,本郷(廿日市),小瀬川,0.31,0.31,0.31,0.32,0.32,0.32,0.31,...,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32
4,0,駄荷(電),太田川,0.39,0.39,0.39,0.39,0.39,0.39,0.39,...,0.39,*,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4


In [6]:
water_stations.head()

,観測所名称,フリガナ,水系名,河川名,事務所,市町,データ所管,住所,緯度,経度,入力時使用,評価対象
0,多治比,タジヒ,江の川,多治比川,西部建設事務所,安芸高田市,河川課,安芸高田市吉田町多冶比,34.681944,132.667222,1,1
1,南原,ナバラ,太田川,南原川,西部建設事務所,広島市安佐北区,河川課,広島市安佐北区可部町上町屋字下小野1452番2,34.540556,132.501389,1,1
2,向原,ムカイハラ,太田川,三篠川,西部建設事務所,安芸高田市,河川課,安芸高田市向原町長田字田屋３４６２－２地先,34.607306,132.703333,1,1
3,上甲立,カミコウダチ,江の川,本村川,西部建設事務所,安芸高田市,河川課,安芸高田市甲田町上甲立字六地蔵１２５２－１地先,34.708333,132.739444,1,1
4,上安,カミヤス,太田川,安川,西部建設事務所,広島市安佐南区,河川課,広島市安佐南区相田２丁目地先,34.469306,132.460000,1,1


In [6]:
water_stations["入力時使用"].value_counts()

1    179
0      1
Name: 入力時使用, dtype: int64

In [7]:
water_stations["評価対象"].value_counts()

1    166
0     14
Name: 評価対象, dtype: int64

In [18]:
water_stations["水系名"].value_counts()

芦田川     46
江の川     39
太田川     34
沼田川     13
単独河川     9
小瀬川      8
黒瀬川      7
八幡川      6
瀬野川      2
藤井川      2
高梁川      2
野呂川      2
賀茂川      2
二河川      2
山南川      1
羽原川      1
西野川      1
御手洗川     1
和久原川     1
本郷川      1
Name: 水系名, dtype: int64

In [19]:
water_stations["河川名"].value_counts()

芦田川    13
太田川     9
小瀬川     8
江の川     8
西城川     7
       ..
吉野川     1
有地川     1
箱田川     1
河手川     1
田総川     1
Name: 河川名, Length: 91, dtype: int64

In [20]:
water_stations["事務所"].value_counts()

西部建設事務所      45
東部建設事務所      41
三原支所         22
北部建設事務所      20
東広島支所        15
廿日市支所        11
安芸太田支所        9
庄原支所          9
呉支所           5
東部建設\n事務所     2
西部建設\n事務所     1
Name: 事務所, dtype: int64

In [21]:
water_stations["市町"].value_counts()

福山市        36
三次市        20
広島市安佐北区    15
東広島市       12
三原市        10
広島市佐伯区     10
庄原市         9
安芸高田市       7
世羅町         7
府中市         7
大竹市         6
安芸太田町       5
尾道市         5
呉市          5
廿日市市        5
北広島町        4
竹原市         3
広島市安佐南区     3
広島市南区       2
広島市中区       2
広島市西区       2
熊野町         1
広島市安芸区      1
府中町         1
海田町         1
広島市東区       1
Name: 市町, dtype: int64

In [9]:
water_stations["データ所管"].value_counts()

河川課         92
国土交通省       46
河川課\nダムG    26
国ダム         12
農業基盤課        4
Name: データ所管, dtype: int64

In [7]:
pd.set_option("display.max_columns", 30)

In [8]:
# 大谷池の水位の急激な変化、その１
df_ohtani_1 = water_data[(water_data["station"]=="大谷池") & (water_data["date"]>= 570) &(water_data["date"]<= 590)]
df_ohtani_1

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
102187,570,大谷池,四川,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
102366,571,大谷池,四川,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.14,*,305.14,305.14,*,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.14,*,305.14,305.14,305.14
102545,572,大谷池,四川,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.14,*,305.14,305.14,305.14,305.15,305.15,305.15,305.15,305.15,305.14,305.14,305.14,305.14,305.14
102724,573,大谷池,四川,305.14,305.14,*,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.14,305.15,*,*,*,*,*,305.15
102903,574,大谷池,四川,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
103082,575,大谷池,四川,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,*,305.15,305.15,305.15,305.15,305.14,*,*,305.14,305.14,305.14
103261,576,大谷池,四川,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
103440,577,大谷池,四川,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.15,305.14,305.14,305.14,305.14,305.15,305.15
103619,578,大谷池,四川,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
103798,579,大谷池,四川,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*


In [15]:
# 大谷池の水位の急激な変化、その２
df_ohtani_2 = water_data[(water_data["station"]=="大谷池") & (water_data["date"]>= 1290) &(water_data["date"]<= 1310)]
df_ohtani_2

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
231067,1290,大谷池,四川,204.54,204.53,204.52,204.51,204.5,204.49,204.48,204.47,204.46,204.45,204.44,204.43,204.42,204.4,204.39,204.38,204.37,204.36,204.35,204.34,204.33,204.32,204.31,204.29
231246,1291,大谷池,四川,204.28,204.27,204.26,204.25,204.24,204.23,204.22,204.21,204.2,204.19,204.17,204.16,204.15,204.14,204.13,204.12,204.11,204.1,204.09,204.08,204.06,204.05,204.04,204.03
231425,1292,大谷池,四川,204.02,204.01,204,203.99,203.98,203.97,203.96,203.95,203.94,203.92,203.91,203.9,203.89,203.88,203.87,203.86,203.85,203.84,203.83,203.82,203.81,203.8,203.78,203.77
231604,1293,大谷池,四川,203.76,203.75,203.74,203.73,203.72,203.71,203.7,203.69,203.68,203.67,203.66,203.65,203.64,203.63,203.62,203.61,203.6,203.59,203.58,203.57,203.56,203.55,203.54,203.53
231783,1294,大谷池,四川,203.52,203.51,203.5,203.49,203.48,203.47,203.46,203.45,203.44,203.43,203.43,203.42,203.41,203.4,203.4,203.39,203.38,203.37,203.36,203.35,203.34,203.33,203.33,203.32
231962,1295,大谷池,四川,203.31,203.3,203.29,203.28,203.27,203.27,203.26,203.25,203.24,203.23,203.22,203.21,203.2,203.2,203.19,203.18,203.17,203.16,203.15,203.14,203.14,203.13,203.12,203.11
232141,1296,大谷池,四川,203.1,203.09,203.09,203.08,203.07,203.06,203.05,203.05,203.04,203.03,203.02,203,202.99,202.98,202.97,202.96,202.95,202.93,202.92,202.91,202.89,202.88,202.87,202.86
232320,1297,大谷池,四川,202.84,202.83,202.82,202.81,202.8,202.79,202.78,202.76,202.75,202.74,202.73,202.72,202.71,202.7,202.69,202.68,202.67,202.66,202.65,202.64,202.63,202.62,202.61,202.6
232499,1298,大谷池,四川,202.59,202.58,202.57,202.56,202.55,202.54,202.53,202.52,202.51,202.5,202.49,202.48,202.48,202.47,202.47,202.46,202.45,202.45,202.45,202.44,202.44,202.43,202.43,202.42
232678,1299,大谷池,四川,202.42,202.42,202.41,202.41,202.41,202.4,202.4,202.39,202.39,202.39,202.38,202.38,202.38,202.37,202.37,202.37,202.37,202.36,202.36,202.36,202.35,202.35,202.35,202.35


=> dateが586の00:00:00から1303の12:00:00まで水位が100m下がっている

In [9]:
# 白川の水位の急激な変化
df_shirakawa = water_data[(water_data["station"]=="白川") & (water_data["date"]>= 680) &(water_data["date"]<= 700)]
df_shirakawa

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
121756,680,白川,八幡川,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07,228.07
121935,681,白川,八幡川,228.07,228.07,228.07,228.07,228.07,228.06,228.06,228.06,228.06,228.06,228.06,228.07,228.08,228.09,228.11,228.12,228.13,228.13,228.15,228.15,228.14,228.18,228.27,228.29
122114,682,白川,八幡川,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
122293,683,白川,八幡川,228.16,228.15,228.14,228.14,228.13,228.13,228.13,228.12,228.12,228.12,228.12,228.12,228.12,228.12,228.11,228.11,228.11,228.11,228.11,228.11,228.1,228.1,228.1,228.1
122472,684,白川,八幡川,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
122651,685,白川,八幡川,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
122830,686,白川,八幡川,228.28,228.27,228.25,228.24,228.23,228.22,228.23,228.28,228.41,228.36,228.4,228.52,228.46,228.44,228.42,228.43,228.42,228.42,228.38,228.39,228.41,228.39,228.36,228.34
123009,687,白川,八幡川,228.32,228.31,228.3,228.29,228.29,228.28,228.28,228.27,228.26,228.26,228.26,228.25,228.25,228.24,228.24,228.23,228.23,228.23,228.22,228.22,228.22,228.21,228.21,228.21
123188,688,白川,八幡川,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
123367,689,白川,八幡川,228.16,228.16,228.16,228.16,228.16,228.16,228.16,228.16,228.16,228.15,228.15,-,228.15,228.15,228.15,228.15,228.15,228.15,228.15,228.14,228.14,228.14,228.14,228.14


=> dateが698の15:00:00のタイミングで、水位が227.5m下がり、そのまま続く

In [3]:
# 評価対象に絞る
L = list(water_stations[water_stations["評価対象"]==1]["観測所名称"])
water_data2 = water_data[water_data["station"].isin(L)]
water_data2_cp = water_data2.copy()
for i in range(24):
    water_data2_cp[f'{str(i).zfill(2)}:00:00'] = pd.to_numeric(water_data2_cp[f'{str(i).zfill(2)}:00:00'], errors='coerce')

In [4]:
water_data2_cp.head()

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
0,0,釜ヶ原,小瀬川,0.76,0.76,0.76,0.76,0.76,0.76,0.76,...,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76
1,0,下ヶ原(国),小瀬川,0.71,0.71,0.71,0.71,0.71,0.71,0.71,...,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71
2,0,防鹿(国),小瀬川,4.71,4.71,4.70,4.70,4.71,4.71,4.71,...,4.70,4.70,4.71,4.70,4.70,4.70,4.71,4.70,4.70,4.70
3,0,本郷(廿日市),小瀬川,0.31,0.31,0.31,0.32,0.32,0.32,0.31,...,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32
5,0,岩倉,小瀬川,0.27,0.27,0.27,0.27,0.27,0.27,0.27,...,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27


In [5]:
# 大谷池の値変更
for i in range(24):
    water_data2_cp.loc[(water_data2_cp["station"]=="大谷池") & (586<=water_data2_cp["date"]) & (water_data2_cp["date"]<=1302), f'{str(i).zfill(2)}:00:00'] += 100
    
# 大谷池の値変更その2
for i in range(13):
    water_data2_cp.loc[(water_data2_cp["station"]=="大谷池") & (water_data2_cp["date"]==1303), f'{str(i).zfill(2)}:00:00'] += 100

In [6]:
#確認
water_data2_cp[(water_data2_cp["station"]=="大谷池") & (water_data2_cp["date"]>= 570) &(water_data2_cp["date"]<= 590)]

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
102187,570,大谷池,四川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102366,571,大谷池,四川,305.14,305.14,305.14,305.14,305.14,305.14,305.14,...,305.14,305.14,305.14,305.14,305.14,305.14,NaN,305.14,305.14,305.14
102545,572,大谷池,四川,305.14,305.14,305.14,305.14,305.14,305.14,305.14,...,305.15,305.15,305.15,305.15,305.15,305.14,305.14,305.14,305.14,305.14
102724,573,大谷池,四川,305.14,305.14,NaN,305.14,305.14,305.14,305.14,...,305.14,305.14,305.14,305.15,NaN,NaN,NaN,NaN,NaN,305.15
102903,574,大谷池,四川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103082,575,大谷池,四川,305.15,305.15,305.15,305.15,305.15,305.15,305.15,...,305.15,305.15,305.15,305.15,305.14,NaN,NaN,305.14,305.14,305.14
103261,576,大谷池,四川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103440,577,大谷池,四川,305.15,305.15,305.15,305.15,305.15,305.15,305.15,...,305.15,305.15,305.15,305.15,305.14,305.14,305.14,305.14,305.15,305.15
103619,578,大谷池,四川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103798,579,大谷池,四川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#確認
water_data2_cp[(water_data2_cp["station"]=="大谷池") & (water_data2_cp["date"]>= 1290) &(water_data2_cp["date"]<= 1310)]

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
231067,1290,大谷池,四川,304.54,304.53,304.52,304.51,304.50,304.49,304.48,304.47,304.46,304.45,304.44,304.43,304.42,304.40,304.39,304.38,304.37,304.36,304.35,304.34,304.33,304.32,304.31,304.29
231246,1291,大谷池,四川,304.28,304.27,304.26,304.25,304.24,304.23,304.22,304.21,304.20,304.19,304.17,304.16,304.15,304.14,304.13,304.12,304.11,304.10,304.09,304.08,304.06,304.05,304.04,304.03
231425,1292,大谷池,四川,304.02,304.01,304.00,303.99,303.98,303.97,303.96,303.95,303.94,303.92,303.91,303.90,303.89,303.88,303.87,303.86,303.85,303.84,303.83,303.82,303.81,303.80,303.78,303.77
231604,1293,大谷池,四川,303.76,303.75,303.74,303.73,303.72,303.71,303.70,303.69,303.68,303.67,303.66,303.65,303.64,303.63,303.62,303.61,303.60,303.59,303.58,303.57,303.56,303.55,303.54,303.53
231783,1294,大谷池,四川,303.52,303.51,303.50,303.49,303.48,303.47,303.46,303.45,303.44,303.43,303.43,303.42,303.41,303.40,303.40,303.39,303.38,303.37,303.36,303.35,303.34,303.33,303.33,303.32
231962,1295,大谷池,四川,303.31,303.30,303.29,303.28,303.27,303.27,303.26,303.25,303.24,303.23,303.22,303.21,303.20,303.20,303.19,303.18,303.17,303.16,303.15,303.14,303.14,303.13,303.12,303.11
232141,1296,大谷池,四川,303.10,303.09,303.09,303.08,303.07,303.06,303.05,303.05,303.04,303.03,303.02,303.00,302.99,302.98,302.97,302.96,302.95,302.93,302.92,302.91,302.89,302.88,302.87,302.86
232320,1297,大谷池,四川,302.84,302.83,302.82,302.81,302.80,302.79,302.78,302.76,302.75,302.74,302.73,302.72,302.71,302.70,302.69,302.68,302.67,302.66,302.65,302.64,302.63,302.62,302.61,302.60
232499,1298,大谷池,四川,302.59,302.58,302.57,302.56,302.55,302.54,302.53,302.52,302.51,302.50,302.49,302.48,302.48,302.47,302.47,302.46,302.45,302.45,302.45,302.44,302.44,302.43,302.43,302.42
232678,1299,大谷池,四川,302.42,302.42,302.41,302.41,302.41,302.40,302.40,302.39,302.39,302.39,302.38,302.38,302.38,302.37,302.37,302.37,302.37,302.36,302.36,302.36,302.35,302.35,302.35,302.35


In [14]:
water_data2_cp[water_data2_cp["date"]==1000]

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
179000,1000,釜ヶ原,小瀬川,0.96,0.96,0.95,0.95,0.95,0.94,0.94,0.93,0.93,0.92,0.92,0.90,0.89,0.89,0.89,0.89,0.89,0.89,0.90,0.90,0.90,0.89,0.89,0.89
179001,1000,下ヶ原(国),小瀬川,1.05,1.04,1.03,1.03,1.03,1.02,1.02,1.02,1.01,1.01,1.00,0.99,0.98,0.98,0.98,0.97,0.97,0.97,0.98,0.98,0.98,0.97,0.97,0.97
179002,1000,防鹿(国),小瀬川,4.89,4.89,4.89,4.89,4.89,4.89,4.89,4.89,4.89,4.88,4.89,4.89,4.88,4.89,4.89,4.88,4.89,4.88,4.88,4.85,4.84,4.83,4.83,4.83
179003,1000,本郷(廿日市),小瀬川,0.55,0.54,0.54,0.53,0.53,0.52,0.52,0.51,0.51,0.51,0.51,0.50,0.50,0.50,0.50,0.50,0.49,0.49,0.49,0.49,0.49,0.49,0.49,0.49
179005,1000,岩倉,小瀬川,0.46,0.46,0.46,0.46,0.46,0.46,0.46,0.46,0.45,0.45,0.45,0.45,0.45,0.45,0.45,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179174,1000,山手(国),芦田川,0.75,0.76,0.77,0.77,0.76,0.75,0.73,0.69,0.65,0.63,0.60,0.58,0.57,0.57,0.57,0.57,0.57,0.57,0.56,0.55,0.53,0.52,0.52,0.51
179175,1000,山手左岸(国),芦田川,0.76,0.77,0.77,0.77,0.75,0.74,0.73,0.70,0.66,0.64,0.59,0.55,0.56,0.57,0.57,0.57,0.56,0.56,0.55,0.54,0.53,0.52,0.51,0.51
179176,1000,今津,本郷川,-0.22,-0.23,-0.24,-0.25,-0.25,-0.26,-0.27,-0.27,-0.28,-0.28,-0.29,-0.30,-0.30,-0.31,-0.32,-0.32,-0.33,-0.33,-0.34,-0.34,-0.34,-0.35,-0.35,-0.35
179177,1000,松永,羽原川,0.47,0.47,0.46,0.46,0.45,0.45,0.45,0.44,0.44,0.44,0.43,0.43,0.42,0.42,0.41,0.41,0.41,0.40,0.40,0.40,0.40,0.39,0.39,0.39


In [7]:
# 白川の値変更
for i in range(24):
    water_data2_cp.loc[(water_data2_cp["station"]=="白川") & (water_data2_cp["date"]<=697), f'{str(i).zfill(2)}:00:00'] -= 227.5
    
# 変更その2
for i in range(15):
    water_data2_cp.loc[(water_data2_cp["station"]=="白川") & (water_data2_cp["date"]==698), f'{str(i).zfill(2)}:00:00'] -= 227.5

In [16]:
#確認
water_data2_cp[(water_data2_cp["station"]=="白川") & (water_data2_cp["date"]>= 690) &(water_data2_cp["date"]<= 710)]

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
123546,690,白川,八幡川,0.64,0.64,0.64,0.64,0.64,0.64,0.64,0.64,0.63,0.63,0.63,0.63,0.63,0.63,0.63,0.63,0.63,0.63,0.63,0.63,0.63,0.63,0.63,0.63
123725,691,白川,八幡川,0.63,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62
123904,692,白川,八幡川,0.62,0.62,0.63,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61
124083,693,白川,八幡川,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.62,0.62,0.63,0.63,0.62,0.62,0.62,0.61,0.61,0.61,0.61
124262,694,白川,八幡川,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60
124441,695,白川,八幡川,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.61,0.61,0.61,0.62,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.61,0.61,0.60
124620,696,白川,八幡川,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.59
124799,697,白川,八幡川,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59
124978,698,白川,八幡川,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59
125157,699,白川,八幡川,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58


In [17]:
water_data2_cp[water_data2_cp["date"]==300]

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
53700,300,釜ヶ原,小瀬川,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73,0.73
53701,300,下ヶ原(国),小瀬川,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.70,0.70,0.70,0.70,0.70,0.70
53702,300,防鹿(国),小瀬川,4.72,4.72,4.72,4.72,4.72,4.72,4.72,4.72,4.72,4.72,4.72,4.72,4.71,4.71,4.72,4.71,4.71,4.71,4.71,4.71,4.71,4.71,4.71,4.71
53703,300,本郷(廿日市),小瀬川,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.30
53705,300,岩倉,小瀬川,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53874,300,山手(国),芦田川,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
53875,300,山手左岸(国),芦田川,0.16,0.16,0.16,0.16,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.14,0.14,0.14,0.16,0.15,0.15,0.15,0.14,0.14,0.14,0.14,0.14
53876,300,今津,本郷川,-0.58,-0.58,-0.58,-0.58,-0.59,-0.59,-0.59,-0.59,-0.59,-0.59,-0.59,-0.59,-0.59,-0.59,-0.59,-0.59,-0.59,-0.59,-0.60,-0.60,-0.60,-0.60,-0.60,-0.60
53877,300,松永,羽原川,0.17,0.17,0.17,0.16,0.16,0.16,0.16,0.16,0.16,0.16,0.16,0.16,0.16,0.16,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15


In [8]:
water_data2['mean'] = water_data2_cp[water_data2_cp.columns[3:]].mean(axis=1)
water_data2['max'] = water_data2_cp[water_data2_cp.columns[3:]].max(axis=1)
water_data2['min'] = water_data2_cp[water_data2_cp.columns[3:]].min(axis=1)
water_data2['std'] = water_data2_cp[water_data2_cp.columns[3:]].std(axis=1)
water_data2['h-0'] = water_data2_cp["00:00:00"]
water_data2['h-23'] = water_data2_cp["23:00:00"]
for i in range(24):
    water_data2 = water_data2.drop(columns=[f'{str(i).zfill(2)}:00:00'])
water_data2.head()

/tmp/ipykernel_363/799590773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  water_data2['mean'] = water_data2_cp[water_data2_cp.columns[3:]].mean(axis=1)
/tmp/ipykernel_363/799590773.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  water_data2['max'] = water_data2_cp[water_data2_cp.columns[3:]].max(axis=1)
/tmp/ipykernel_363/799590773.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

,date,station,river,mean,max,min,std,h-0,h-23
0,0,釜ヶ原,小瀬川,0.760000,0.76,0.76,1.134102e-16,0.76,0.76
1,0,下ヶ原(国),小瀬川,0.710000,0.71,0.71,0.000000e+00,0.71,0.71
2,0,防鹿(国),小瀬川,4.703333,4.71,4.70,4.815434e-03,4.71,4.70
3,0,本郷(廿日市),小瀬川,0.317083,0.32,0.31,4.643056e-03,0.31,0.32
5,0,岩倉,小瀬川,0.270000,0.27,0.27,0.000000e+00,0.27,0.27


In [9]:
# 1日前
water_data2_1d_ago = water_data2.copy()
for name in water_data2.columns:
    if name not in ["date", "station", "river"]:
        water_data2_1d_ago.rename(columns={name: name+"_1d_ago"}, inplace=True)
water_data2_1d_ago.head()

,date,station,river,mean_1d_ago,max_1d_ago,min_1d_ago,std_1d_ago,h-0_1d_ago,h-23_1d_ago
0,0,釜ヶ原,小瀬川,0.760000,0.76,0.76,1.134102e-16,0.76,0.76
1,0,下ヶ原(国),小瀬川,0.710000,0.71,0.71,0.000000e+00,0.71,0.71
2,0,防鹿(国),小瀬川,4.703333,4.71,4.70,4.815434e-03,4.71,4.70
3,0,本郷(廿日市),小瀬川,0.317083,0.32,0.31,4.643056e-03,0.31,0.32
5,0,岩倉,小瀬川,0.270000,0.27,0.27,0.000000e+00,0.27,0.27


In [10]:
water_data2_1d_ago["date"] = water_data2_1d_ago["date"] + 1

In [11]:
# 2日前
water_data2_2d_ago = water_data2.copy()
for name in water_data2.columns:
    if name not in ["date", "station", "river"]:
        water_data2_2d_ago.rename(columns={name: name+"_2d_ago"}, inplace=True)
water_data2_2d_ago["date"] = water_data2_2d_ago["date"] + 2

# 3日前
water_data2_3d_ago = water_data2.copy()
for name in water_data2.columns:
    if name not in ["date", "station", "river"]:
        water_data2_3d_ago.rename(columns={name: name+"_3d_ago"}, inplace=True)
water_data2_3d_ago["date"] = water_data2_3d_ago["date"] + 3

# 4日前
water_data2_4d_ago = water_data2.copy()
for name in water_data2.columns:
    if name not in ["date", "station", "river"]:
        water_data2_4d_ago.rename(columns={name: name+"_4d_ago"}, inplace=True)
water_data2_4d_ago["date"] = water_data2_4d_ago["date"] + 4


# 5日前
water_data2_5d_ago = water_data2.copy()
for name in water_data2.columns:
    if name not in ["date", "station", "river"]:
        water_data2_5d_ago.rename(columns={name: name+"_5d_ago"}, inplace=True)
water_data2_5d_ago["date"] = water_data2_5d_ago["date"] + 5

In [12]:
water_data2_5d_ago

,date,station,river,mean_5d_ago,max_5d_ago,min_5d_ago,std_5d_ago,h-0_5d_ago,h-23_5d_ago
0,5,釜ヶ原,小瀬川,0.760000,0.76,0.76,1.134102e-16,0.76,0.76
1,5,下ヶ原(国),小瀬川,0.710000,0.71,0.71,0.000000e+00,0.71,0.71
2,5,防鹿(国),小瀬川,4.703333,4.71,4.70,4.815434e-03,4.71,4.70
3,5,本郷(廿日市),小瀬川,0.317083,0.32,0.31,4.643056e-03,0.31,0.32
5,5,岩倉,小瀬川,0.270000,0.27,0.27,0.000000e+00,0.27,0.27
...,...,...,...,...,...,...,...,...,...
392061,2195,町田,黒瀬川,0.301667,0.42,0.28,3.472459e-02,0.42,0.28
392062,2195,松ヶ瀬,黒瀬川,0.017500,0.03,0.01,7.372098e-03,0.03,0.01
392063,2195,下見,黒瀬川,0.117083,0.12,0.11,4.643056e-03,0.12,0.11
392064,2195,御薗宇,黒瀬川,0.320417,0.35,0.30,1.517411e-02,0.35,0.32


In [13]:
df_water = pd.merge(water_data2, water_data2_1d_ago, on=["date", "station", "river"], how="left")
df_water

,date,station,river,mean,max,min,std,h-0,h-23,mean_1d_ago,max_1d_ago,min_1d_ago,std_1d_ago,h-0_1d_ago,h-23_1d_ago
0,0,釜ヶ原,小瀬川,0.760000,0.76,0.76,1.134102e-16,0.76,0.76,NaN,NaN,NaN,NaN,NaN,NaN
1,0,下ヶ原(国),小瀬川,0.710000,0.71,0.71,0.000000e+00,0.71,0.71,NaN,NaN,NaN,NaN,NaN,NaN
2,0,防鹿(国),小瀬川,4.703333,4.71,4.70,4.815434e-03,4.71,4.70,NaN,NaN,NaN,NaN,NaN,NaN
3,0,本郷(廿日市),小瀬川,0.317083,0.32,0.31,4.643056e-03,0.31,0.32,NaN,NaN,NaN,NaN,NaN,NaN
4,0,岩倉,小瀬川,0.270000,0.27,0.27,0.000000e+00,0.27,0.27,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362651,2190,町田,黒瀬川,0.301667,0.42,0.28,3.472459e-02,0.42,0.28,0.350417,0.42,0.28,0.066821,0.28,0.42
362652,2190,松ヶ瀬,黒瀬川,0.017500,0.03,0.01,7.372098e-03,0.03,0.01,0.035833,0.07,0.00,0.019982,0.00,0.03
362653,2190,下見,黒瀬川,0.117083,0.12,0.11,4.643056e-03,0.12,0.11,0.183750,0.28,0.11,0.060203,0.11,0.12
362654,2190,御薗宇,黒瀬川,0.320417,0.35,0.30,1.517411e-02,0.35,0.32,0.346667,0.41,0.29,0.036316,0.29,0.35


In [14]:
df_water = pd.merge(df_water, water_data2_2d_ago, on=["date", "station", "river"], how="left")
df_water = pd.merge(df_water, water_data2_3d_ago, on=["date", "station", "river"], how="left")
df_water = pd.merge(df_water, water_data2_4d_ago, on=["date", "station", "river"], how="left")
df_water = pd.merge(df_water, water_data2_5d_ago, on=["date", "station", "river"], how="left")
df_water

,date,station,river,mean,max,min,std,h-0,h-23,mean_1d_ago,...,min_4d_ago,std_4d_ago,h-0_4d_ago,h-23_4d_ago,mean_5d_ago,max_5d_ago,min_5d_ago,std_5d_ago,h-0_5d_ago,h-23_5d_ago
0,0,釜ヶ原,小瀬川,0.760000,0.76,0.76,1.134102e-16,0.76,0.76,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,下ヶ原(国),小瀬川,0.710000,0.71,0.71,0.000000e+00,0.71,0.71,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,防鹿(国),小瀬川,4.703333,4.71,4.70,4.815434e-03,4.71,4.70,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,本郷(廿日市),小瀬川,0.317083,0.32,0.31,4.643056e-03,0.31,0.32,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,岩倉,小瀬川,0.270000,0.27,0.27,0.000000e+00,0.27,0.27,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362651,2190,町田,黒瀬川,0.301667,0.42,0.28,3.472459e-02,0.42,0.28,0.350417,...,0.22,0.035834,0.30,0.28,0.272083,0.46,0.09,0.079672,0.11,0.30
362652,2190,松ヶ瀬,黒瀬川,0.017500,0.03,0.01,7.372098e-03,0.03,0.01,0.035833,...,0.01,0.014390,0.06,0.02,0.046667,0.06,0.01,0.015511,0.01,0.06
362653,2190,下見,黒瀬川,0.117083,0.12,0.11,4.643056e-03,0.12,0.11,0.183750,...,0.11,0.017425,0.17,0.11,0.194583,0.27,0.12,0.042012,0.21,0.19
362654,2190,御薗宇,黒瀬川,0.320417,0.35,0.30,1.517411e-02,0.35,0.32,0.346667,...,0.26,0.016545,0.33,0.33,0.346667,0.44,0.28,0.041037,0.39,0.33


In [17]:
df_water.columns

Index(['date', 'station', 'river', 'mean', 'max', 'min', 'std', 'h-0', 'h-23',
       'mean_1d_ago', 'max_1d_ago', 'min_1d_ago', 'std_1d_ago', 'h-0_1d_ago',
       'h-23_1d_ago', 'mean_2d_ago', 'max_2d_ago', 'min_2d_ago', 'std_2d_ago',
       'h-0_2d_ago', 'h-23_2d_ago', 'mean_3d_ago', 'max_3d_ago', 'min_3d_ago',
       'std_3d_ago', 'h-0_3d_ago', 'h-23_3d_ago', 'mean_4d_ago', 'max_4d_ago',
       'min_4d_ago', 'std_4d_ago', 'h-0_4d_ago', 'h-23_4d_ago', 'mean_5d_ago',
       'max_5d_ago', 'min_5d_ago', 'std_5d_ago', 'h-0_5d_ago', 'h-23_5d_ago'],
      dtype='object')

In [16]:
df_water.to_csv("../data/process-data/water-process-5daysago-修正済み.csv", index = False)

In [52]:
df_water[(df_water["station"]=="大谷池") & (df_water["date"]>= 570) &(df_water["date"]<= 590)]

,date,station,river,mean,max,min,std,h-0,h-23,mean_1d_ago,max_1d_ago,min_1d_ago,std_1d_ago,h-0_1d_ago,h-23_1d_ago,mean_2d_ago,max_2d_ago,min_2d_ago,std_2d_ago,h-0_2d_ago,h-23_2d_ago,mean_3d_ago,max_3d_ago,min_3d_ago,std_3d_ago,h-0_3d_ago,h-23_3d_ago
94764,570,大谷池,四川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,305.141304,305.16,305.14,0.004577,305.14,305.16
94930,571,大谷池,四川,305.140000,305.14,305.14,0.000000,305.14,305.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95096,572,大谷池,四川,305.142174,305.15,305.14,0.004217,305.14,305.14,305.140000,305.14,305.14,0.000000,305.14,305.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95262,573,大谷池,四川,305.141111,305.15,305.14,0.003234,305.14,305.15,305.142174,305.15,305.14,0.004217,305.14,305.14,305.140000,305.14,305.14,0.000000,305.14,305.14,NaN,NaN,NaN,NaN,NaN,NaN
95428,574,大谷池,四川,NaN,NaN,NaN,NaN,NaN,NaN,305.141111,305.15,305.14,0.003234,305.14,305.15,305.142174,305.15,305.14,0.004217,305.14,305.14,305.140000,305.14,305.14,0.000000,305.14,305.14
95594,575,大谷池,四川,305.148095,305.15,305.14,0.004024,305.15,305.14,NaN,NaN,NaN,NaN,NaN,NaN,305.141111,305.15,305.14,0.003234,305.14,305.15,305.142174,305.15,305.14,0.004217,305.14,305.14
95760,576,大谷池,四川,NaN,NaN,NaN,NaN,NaN,NaN,305.148095,305.15,305.14,0.004024,305.15,305.14,NaN,NaN,NaN,NaN,NaN,NaN,305.141111,305.15,305.14,0.003234,305.14,305.15
95926,577,大谷池,四川,305.148333,305.15,305.14,0.003807,305.15,305.15,NaN,NaN,NaN,NaN,NaN,NaN,305.148095,305.15,305.14,0.004024,305.15,305.14,NaN,NaN,NaN,NaN,NaN,NaN
96092,578,大谷池,四川,NaN,NaN,NaN,NaN,NaN,NaN,305.148333,305.15,305.14,0.003807,305.15,305.15,NaN,NaN,NaN,NaN,NaN,NaN,305.148095,305.15,305.14,0.004024,305.15,305.14
96258,579,大谷池,四川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,305.148333,305.15,305.14,0.003807,305.15,305.15,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df_water[(df_water["station"]=="白川") & (df_water["date"]>= 690) &(df_water["date"]<= 710)]

,date,station,river,mean,max,min,std,h-0,h-23,mean_1d_ago,max_1d_ago,min_1d_ago,std_1d_ago,h-0_1d_ago,h-23_1d_ago,mean_2d_ago,max_2d_ago,min_2d_ago,std_2d_ago,h-0_2d_ago,h-23_2d_ago,mean_3d_ago,max_3d_ago,min_3d_ago,std_3d_ago,h-0_3d_ago,h-23_3d_ago
114572,690,白川,八幡川,0.633333,0.64,0.63,4.815434e-03,0.64,0.63,0.651739,0.66,0.64,7.776523e-03,0.66,0.64,NaN,NaN,NaN,NaN,NaN,NaN,0.753333,0.82,0.71,3.318809e-02,0.82,0.71
114738,691,白川,八幡川,0.620417,0.63,0.62,2.041241e-03,0.63,0.62,0.633333,0.64,0.63,4.815434e-03,0.64,0.63,0.651739,0.66,0.64,7.776523e-03,0.66,0.64,NaN,NaN,NaN,NaN,NaN,NaN
114904,692,白川,八幡川,0.615000,0.63,0.61,5.897678e-03,0.62,0.61,0.620417,0.63,0.62,2.041241e-03,0.63,0.62,0.633333,0.64,0.63,4.815434e-03,0.64,0.63,0.651739,0.66,0.64,7.776523e-03,0.66,0.64
115070,693,白川,八幡川,0.613750,0.63,0.61,6.468990e-03,0.61,0.61,0.615000,0.63,0.61,5.897678e-03,0.62,0.61,0.620417,0.63,0.62,2.041241e-03,0.63,0.62,0.633333,0.64,0.63,4.815434e-03,0.64,0.63
115236,694,白川,八幡川,0.604583,0.61,0.60,5.089774e-03,0.61,0.60,0.613750,0.63,0.61,6.468990e-03,0.61,0.61,0.615000,0.63,0.61,5.897678e-03,0.62,0.61,0.620417,0.63,0.62,2.041241e-03,0.63,0.62
115402,695,白川,八幡川,0.607500,0.62,0.60,7.939992e-03,0.60,0.60,0.604583,0.61,0.60,5.089774e-03,0.61,0.60,0.613750,0.63,0.61,6.468990e-03,0.61,0.61,0.615000,0.63,0.61,5.897678e-03,0.62,0.61
115568,696,白川,八幡川,0.598333,0.60,0.59,3.806935e-03,0.60,0.59,0.607500,0.62,0.60,7.939992e-03,0.60,0.60,0.604583,0.61,0.60,5.089774e-03,0.61,0.60,0.613750,0.63,0.61,6.468990e-03,0.61,0.61
115734,697,白川,八幡川,0.590000,0.59,0.59,0.000000e+00,0.59,0.59,0.598333,0.60,0.59,3.806935e-03,0.60,0.59,0.607500,0.62,0.60,7.939992e-03,0.60,0.60,0.604583,0.61,0.60,5.089774e-03,0.61,0.60
115900,698,白川,八幡川,0.590000,0.59,0.59,1.702569e-15,0.59,0.59,0.590000,0.59,0.59,0.000000e+00,0.59,0.59,0.598333,0.60,0.59,3.806935e-03,0.60,0.59,0.607500,0.62,0.60,7.939992e-03,0.60,0.60
116066,699,白川,八幡川,0.581250,0.59,0.58,3.378320e-03,0.59,0.58,0.590000,0.59,0.59,1.702569e-15,0.59,0.59,0.590000,0.59,0.59,0.000000e+00,0.59,0.59,0.598333,0.60,0.59,3.806935e-03,0.60,0.59


In [26]:
# train4.csvも値変更したいので別のDataFrame作成
water_data2_cp

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
0,0,釜ヶ原,小瀬川,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76
1,0,下ヶ原(国),小瀬川,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71
2,0,防鹿(国),小瀬川,4.71,4.71,4.70,4.70,4.71,4.71,4.71,4.70,4.70,4.70,4.70,4.70,4.70,4.71,4.70,4.70,4.71,4.70,4.70,4.70,4.71,4.70,4.70,4.70
3,0,本郷(廿日市),小瀬川,0.31,0.31,0.31,0.32,0.32,0.32,0.31,0.31,0.31,0.31,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32
5,0,岩倉,小瀬川,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392061,2190,町田,黒瀬川,0.42,0.40,0.33,0.30,0.30,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.29,0.28,0.28
392062,2190,松ヶ瀬,黒瀬川,0.03,0.03,0.02,0.02,0.02,0.03,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
392063,2190,下見,黒瀬川,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.11,0.11,0.11,0.11,0.11,0.11,0.11
392064,2190,御薗宇,黒瀬川,0.35,0.35,0.34,0.34,0.34,0.34,0.32,0.30,0.31,0.32,0.30,0.32,0.31,0.31,0.31,0.31,0.31,0.31,0.30,0.32,0.32,0.32,0.32,0.32


In [41]:
# 1時間ごとのデータに
df_water_per_hour = pd.DataFrame(water_data2_cp.set_index(["date", "station", "river"]).stack(dropna=False)).reset_index().rename(columns={0:"water level", "level_3":"hour"})

In [42]:
df_water_per_hour["hour"] = df_water_per_hour["hour"].str[:2].astype(int)
df_water_per_hour["hour_sin"] = np.sin(np.pi*df_water_per_hour["hour"]/12)
df_water_per_hour["hour_cos"] = np.cos(np.pi*df_water_per_hour["hour"]/12)

In [43]:
df_water_per_hour.head()

,date,station,river,hour,water level,hour_sin,hour_cos
0,0,釜ヶ原,小瀬川,0,0.76,0.000000,1.000000
1,0,釜ヶ原,小瀬川,1,0.76,0.258819,0.965926
2,0,釜ヶ原,小瀬川,2,0.76,0.500000,0.866025
3,0,釜ヶ原,小瀬川,3,0.76,0.707107,0.707107
4,0,釜ヶ原,小瀬川,4,0.76,0.866025,0.500000


In [45]:
df_water_per_hour[(df_water_per_hour["station"]=="大谷池") & (df_water_per_hour["date"]>= 571) &(df_water_per_hour["date"]<= 590)]

,date,station,river,hour,water level,hour_sin,hour_cos
2278320,571,大谷池,四川,0,305.14,0.000000,1.000000
2278321,571,大谷池,四川,1,305.14,0.258819,0.965926
2278322,571,大谷池,四川,2,305.14,0.500000,0.866025
2278323,571,大谷池,四川,3,305.14,0.707107,0.707107
2278324,571,大谷池,四川,4,305.14,0.866025,0.500000
...,...,...,...,...,...,...,...
2354035,590,大谷池,四川,19,NaN,-0.965926,0.258819
2354036,590,大谷池,四川,20,NaN,-0.866025,0.500000
2354037,590,大谷池,四川,21,NaN,-0.707107,0.707107
2354038,590,大谷池,四川,22,NaN,-0.500000,0.866025


In [46]:
df_a = df_water_per_hour[["date", "hour", "station", "water level"]].copy()
df_a["date"] = df_a["date"] - 1
df_a = df_a.rename(columns={'water level': 'next water level'})
df_a.head()

,date,hour,station,next water level
0,-1,0,釜ヶ原,0.76
1,-1,1,釜ヶ原,0.76
2,-1,2,釜ヶ原,0.76
3,-1,3,釜ヶ原,0.76
4,-1,4,釜ヶ原,0.76


In [47]:
df = pd.merge(df_water_per_hour, df_a, on=["date", "hour", "station"], how="left")

In [48]:
df.shape

(8706696, 8)

In [49]:
df = df.dropna(subset=["next water level"])

In [50]:
df.shape

(8183676, 8)

In [51]:
df.head()

,date,station,river,hour,water level,hour_sin,hour_cos,next water level
0,0,釜ヶ原,小瀬川,0,0.76,0.000000,1.000000,0.76
1,0,釜ヶ原,小瀬川,1,0.76,0.258819,0.965926,0.76
2,0,釜ヶ原,小瀬川,2,0.76,0.500000,0.866025,0.76
3,0,釜ヶ原,小瀬川,3,0.76,0.707107,0.707107,0.76
4,0,釜ヶ原,小瀬川,4,0.76,0.866025,0.500000,0.76


In [52]:
df[(df["station"]=="大谷池") & (df["date"]>= 570) &(df["date"]<= 590)]

,date,station,river,hour,water level,hour_sin,hour_cos,next water level
2274336,570,大谷池,四川,0,NaN,0.000000,1.000000,305.14
2274337,570,大谷池,四川,1,NaN,0.258819,0.965926,305.14
2274338,570,大谷池,四川,2,NaN,0.500000,0.866025,305.14
2274339,570,大谷池,四川,3,NaN,0.707107,0.707107,305.14
2274340,570,大谷池,四川,4,NaN,0.866025,0.500000,305.14
...,...,...,...,...,...,...,...,...
2334113,585,大谷池,四川,17,NaN,-0.965926,-0.258819,305.14
2334115,585,大谷池,四川,19,NaN,-0.965926,0.258819,305.14
2334117,585,大谷池,四川,21,NaN,-0.707107,0.707107,305.14
2334118,585,大谷池,四川,22,NaN,-0.500000,0.866025,305.14


In [53]:
df[(df["station"]=="大谷池") & (df["date"]== 585)]

,date,station,river,hour,water level,hour_sin,hour_cos,next water level
2334096,585,大谷池,四川,0,NaN,0.000000e+00,1.000000e+00,305.14
2334097,585,大谷池,四川,1,NaN,2.588190e-01,9.659258e-01,305.14
2334098,585,大谷池,四川,2,NaN,5.000000e-01,8.660254e-01,305.14
2334099,585,大谷池,四川,3,NaN,7.071068e-01,7.071068e-01,305.14
2334100,585,大谷池,四川,4,NaN,8.660254e-01,5.000000e-01,305.14
2334101,585,大谷池,四川,5,NaN,9.659258e-01,2.588190e-01,305.14
2334102,585,大谷池,四川,6,NaN,1.000000e+00,6.123234e-17,305.14
2334103,585,大谷池,四川,7,NaN,9.659258e-01,-2.588190e-01,305.14
2334104,585,大谷池,四川,8,NaN,8.660254e-01,-5.000000e-01,305.14
2334106,585,大谷池,四川,10,NaN,5.000000e-01,-8.660254e-01,305.14


In [54]:
df.to_csv("../data/process-data/train4-修正済み.csv", index = False)

In [55]:
# 相関係数
df[["water level", "next water level", "hour_sin", "hour_cos"]].corr()

,water level,next water level,hour_sin,hour_cos
water level,1.000000,0.999958,-0.000376,-0.001170
next water level,0.999958,1.000000,-0.000207,-0.000660
hour_sin,-0.000376,-0.000207,1.000000,-0.001788
hour_cos,-0.001170,-0.000660,-0.001788,1.000000
